Decode Sequencing Run
-----------------------------

#### Preparation

1. Make sure the docker image was started with `./docker.sh -s /path/to/reads`, where `/path/to/reads` is a directory on the same drive as the docker image, which contains the FASTQ files for your sequencing runs. This directory will be mounted within the running image as `/tf/sequencing`.

2. Make a new directory called `/tf/primo/data/sequencing/run_id` for this run, where `run_id` is up to you. It must contain two files:

    a. `index.csv`: a comma-separated table of metadata for the run. Each row represents an Illumina index (or other identifier) within the run. The first column must be labeled `sequencing_index` and contain this identifier. The rest of the columns are up to you, and contain the properties of the experiment that was given that index.
    
    b. `location`: a combination python format string / wildcard string that points to the location of all of the FASTQ files for each sequencing index in the run. For instance, if sequencing reads are contained in files like:
    ```
    /path/to/reads/Run_105/B11_SSC_8m_10x_1_L001-ds.719452199380440faadf49ed854f0cbb/B11-SSC-8m-10x-1_S3_L001_R1_001.fastq.gz
    ```
    
    then `location` would contain the following:
    
    ```
    /tf/sequencing/Run_105/%s_*/*.gz
    ```

In [1]:
import numpy as np
import pandas as pd
import subprocess
import tempfile
import multiprocessing

from primo.tools.barcoder import Barcoder

This is the internal primer (conserved across all reads) which is immediately downstream of the barcode:

In [2]:
IP = "AGCACTCAGTATTTGTCCG"

This file maps barcodes (which are just sequence numbers 0, 1, 2, etc.) to OpenImages IDs:

In [3]:
barcode_order = pd.read_csv('/tf/primo/data/metadata/target_barcode_order.csv.gz')

These parameters (including the random seed) should be the same used for encoding:

In [4]:
barcoder = Barcoder(
    n_data_symbols   = 4,
    n_check_symbols  = 2,
    bits_per_symbol  = 6,
    bases_per_symbol = 5,
    seed = 42
)
def decode_barcode(barcode):
    return barcoder.barcode_seq_to_num(barcode.strip())

This function searches through the reads for a particular sequencing index, looking for exact matches of the internal primer, and collects the barcodes (which are the 30 bases upstream of the internal primer). It then decodes all of the recovered barcodes and returns the count of successful decodes for each barcode:

In [5]:
def decode_index(path_glob):
    
    with tempfile.NamedTemporaryFile() as temp:
        
        if path_glob.endswith(".gz"):
            cat_cmd = "zcat %s" % path_glob
        else:
            cat_cmd = "cat %s" % path_glob
    
        # extract barcodes
        subprocess.call(
            (cat_cmd + "| egrep -o '[ATCGN]{30}%s' | cut -b 1-30 > %s") % (
                IP,
                temp.name
            ),
            shell = True
        )
        
        barcodes = temp.readlines()
            
    # decode
    pool = multiprocessing.Pool()
    try:
        results = np.array(pool.map(decode_barcode, barcodes))
    finally:
        pool.close()
    
    decoded = results[results != None].astype(int)
    
    counts = np.bincount(decoded, minlength=len(barcode_order))[:len(barcode_order)]
    
    return counts

This loops through all of the sequencing indices in a run and builds a pandas data frame with the results:

In [6]:
def decode_run(run_id):
    run_path = '/tf/primo/data/sequencing/%s/' % run_id
    
    # open run meta
    run_meta = pd.read_csv(run_path + 'index.csv')
    with open(run_path + 'location') as f:
        location = f.readline().strip()
        
    # decode each index
    counts = []
    for ix in run_meta.sequencing_index:
        print (run_id, ix)
        path_glob = location % ix
        counts.append(decode_index(path_glob))

    # save
    df = pd.DataFrame(
       np.array(counts),
       index = run_meta.sequencing_index,
       columns = barcode_order.ImageID
    )
    df.to_pickle(run_path + 'decoded.pkl.gz')
    
    return df

The argument here must be a directory in `/tf/primo/data/sequencing` that contains both `index.csv` and `location`, as described at the start of this notebook:

In [7]:
df = decode_run("Run_107")

('Run_107', 'G7')
('Run_107', 'G11')
('Run_107', 'H7')
('Run_107', 'H9')
('Run_107', 'B9')
('Run_107', 'C1')
('Run_107', 'D1')
('Run_107', 'D10')
('Run_107', 'D12')


In [ ]:
run_meta = (
    pd.read_csv('/tf/primo/data/sequencing/Run_105/index.csv')
)
run_meta = run_meta.set_index(list(run_meta.columns[1:]))

In [ ]:
run_meta.join(
    df[['e39871fd9fd74f55', 'Randomer']].join(df.sum(1).rename('total')),
    on='sequencing_index'
)

In [8]:
df

ImageID,e39871fd9fd74f55,f18b91585c4d3f3e,ede6e66b2fb59aab,ed600d57fcee4f94,ff47e649b23f446d,e17acd05b631d330,efcfa9654f0e99c5,f4124588a82d57be,f7a1ee2daf06b9e5,e91ca52128724d8e,...,Unused,Unused,Unused,Unused,Unused,Unused,Unused,Unused,Unused,Randomer
sequencing_index,,,,,,,,,,,,,,,,,,,,,
G7,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,42328
G11,3,0,0,0,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,15911
H7,0,0,0,0,0,0,7,0,0,2,...,0,0,0,0,0,0,0,0,0,3145
H9,3,0,0,4,0,0,4,1,1,0,...,0,0,0,0,0,0,0,0,0,6518
B9,3,0,0,0,1,0,1,0,2,2,...,0,0,0,0,0,0,0,0,0,837
C1,1,0,0,1,1,2,1,4,0,1,...,0,0,0,0,0,0,0,0,0,5112
D1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,40123
D10,0,0,0,1,0,3,2,0,0,0,...,0,0,0,0,0,0,0,0,0,8532
D12,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2342


In [ ]:
df['Randomer'] / df.sum(1)

In [ ]:
df.values[0,:1600000]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(df.values[5,:1600000], log=True)

In [12]:
barcoder.num_to_barcode_seq(1600042)

'CTGATAGTAGATCATAGATGACACGATGAT'

In [11]:
df[['e39871fd9fd74f55', 'Randomer']]

ImageID,e39871fd9fd74f55,Randomer
sequencing_index,,
G7,16,42328
G11,3,15911
H7,0,3145
H9,3,6518
B9,3,837
C1,1,5112
D1,0,40123
D10,0,8532
D12,0,2342
